In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I

In [2]:
m = ModelIntv()
n_collect = 200

In [3]:
out_path = f'out/dy_free'
pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
r_acf = json.load(open(f'{out_path}/R_ACF_Bg.json', 'r'))

In [4]:
for p in pars_post:
    p.update(r_acf)

In [5]:
results = []


for i, pars in enumerate(pars_post[:n_collect]):
    y1, pars = m.find_baseline(pars, 2022)

    _, ms00, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MU': {'Scale': 0}}, t_end=2031)
    _, ms01, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 0}, 'MU': {'Scale': 1}}, t_end=2031)
    _, ms10, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MU': {'Scale': 0}}, t_end=2031)
    _, ms11, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MU': {'Scale': 1}}, t_end=2031)
    
    inc00 = (ms00.IncR * ms00.Pop).sum() * 0.5
    inc01 = (ms01.IncR * ms01.Pop).sum() * 0.5
    inc10 = (ms10.IncR * ms10.Pop).sum() * 0.5
    inc11 = (ms11.IncR * ms11.Pop).sum() * 0.5
    
    results.append({
        'Key': i,
        'Avt_MU': 1 - inc01 / inc00,
        'Avt_D2D': 1 - inc10 / inc00,
        'Avt_MU+D2D': 1 - inc11 / inc00,
    })
    
results = pd.DataFrame(results)

In [6]:
results.describe()

,Key,Avt_MU,Avt_D2D,Avt_MU+D2D
count,200.000000,200.000000,200.000000,200.000000
mean,99.500000,0.003013,0.013468,0.016385
std,57.879185,0.000283,0.003519,0.003694
min,0.000000,0.002114,0.005146,0.007565
25%,49.750000,0.002828,0.011041,0.013934
50%,99.500000,0.003035,0.013526,0.016374
75%,149.250000,0.003200,0.015944,0.019063
max,199.000000,0.003662,0.024331,0.027810


In [7]:
results.to_csv(f'{out_path}/Sim_BgACF_IncrementalAverted.csv')

In [8]:
idx = ['Pop', 'IncR', 'MorR',
     'Reach_ACF_MU1', 'Reach_ACF_MU2', 'Yield_ACF_MU',
     'Reach_ACF_D2D1', 'Reach_ACF_D2D2', 'Yield_ACF_D2D']

results = []


for i, pars in enumerate(pars_post[:n_collect]):
    y1, pars = m.find_baseline(pars, 2022)

    _, ms1, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 1}, 'MU': {'Scale': 1}}, t_end=2031)
    _, ms2, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 2}, 'MU': {'Scale': 2}}, t_end=2031)
    _, ms4, _ = m.simulate_onward(y1, pars, intv={'D2D': {'Scale': 4}, 'MU': {'Scale': 4}}, t_end=2031)

    results.append(pd.concat([
        ms1[idx].reset_index().assign(Key=i, Scale='1x'),
        ms2[idx].reset_index().assign(Key=i, Scale='2x'),
        ms4[idx].reset_index().assign(Key=i, Scale='4x')
    ]))

results = pd.concat(results)

In [9]:
results.head()

,Time,Pop,IncR,MorR,Reach_ACF_MU1,Reach_ACF_MU2,Yield_ACF_MU,Reach_ACF_D2D1,Reach_ACF_D2D2,Yield_ACF_D2D,Key,Scale
0,2022.0,2.383045,0.005229,0.000753,0.003461,0.000828,0.000042,0.036762,0.004252,0.000213,0,1x
1,2022.5,2.403026,0.005194,0.000742,0.003461,0.000827,0.000041,0.036762,0.004251,0.000212,0,1x
2,2023.0,2.423176,0.005157,0.000735,0.003461,0.000827,0.000041,0.036762,0.004250,0.000211,0,1x
3,2023.5,2.443494,0.005118,0.000729,0.003461,0.000827,0.000041,0.036763,0.004249,0.000210,0,1x
4,2024.0,2.463982,0.005079,0.000722,0.003461,0.000827,0.000041,0.036763,0.004248,0.000209,0,1x


In [10]:
results.to_csv(f'{out_path}/Sim_BgACF_ScaleUp.csv')